In [ ]:
import ch7util
import dautil as dl
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML

In [ ]:
def hl2(df, suffix):
    high = df['High_' + suffix]
    low = df['Low_' + suffix]

    return (high - low) ** 2

In [ ]:
def correlate(stock, sp500):
    merged = ch7util.merge_sp500(stock, sp500)
    rets = ch7util.log_rets(merged['Adj Close_stock'])
    sp500_rets = ch7util.log_rets(merged['Adj Close_sp500'])
    result = {}

    result['corrcoef'] = np.corrcoef(rets, sp500_rets)[0][1]
    slope, _ = np.polyfit(sp500_rets, rets, 1)
    result['slope'] = slope

    srd = (sp500_rets - rets) ** 2
    result['msrd'] = srd.mean()
    result['std_srd'] = srd.std()

    result['vols'] = np.corrcoef(merged['Volume_stock'],
                                 merged['Volume_sp500'])[0][1]

    result['hl2'] = np.corrcoef(hl2(merged, 'stock'),
                                hl2(merged, 'sp500'))[0][1]

    return result

In [ ]:
ohlc = dl.data.OHLC()
dfs = [ohlc.get(stock) for stock in ch7util.STOCKS]
sp500 = ohlc.get('^GSPC')
corrs = [correlate(df, sp500) for df in dfs]

In [ ]:
%matplotlib inline
dl.options.mimic_seaborn()
context = dl.nb.Context('correlating_market')
dl.nb.RcWidget(context)
dl.nb.LabelWidget(2, 2, context)

In [ ]:
sp = dl.plotting.Subplotter(2, 2, context)
dl.plotting.bar(sp.ax, ch7util.STOCKS,
                [corr['corrcoef'] for corr in corrs])
sp.label()

dl.plotting.bar(sp.next_ax(), ch7util.STOCKS,
                [corr['slope'] for corr in corrs])
sp.label()

sp.next_ax().set_xlim([0, 0.001])
dl.plotting.plot_text(sp.ax, [corr['msrd'] for corr in corrs],
                      [corr['std_srd'] for corr in corrs],
                      ch7util.STOCKS, add_scatter=True,
                      fontsize=9, alpha=0.6)
sp.label()

dl.plotting.plot_text(sp.next_ax(), [corr['vols'] for corr in corrs],
                      [corr['hl2'] for corr in corrs],
                      ch7util.STOCKS, add_scatter=True,
                      fontsize=9, alpha=0.6)
sp.label()


HTML(sp.exit())